<a href="https://colab.research.google.com/github/timothyow/review-analyst-llm-01/blob/main/analyst_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project - creating an AI agent (review analyst) using LangChain and LLM**

In [2]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install numpy
!pip install pandas
!pip install openai

In [3]:
import os
import pandas as pd
import numpy as np
from typing import List
from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [4]:
path = "/content/review_data.csv"
df = pd.read_csv(path)
df.head()

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,9334422659,3,0,NaN,0,0,0,3/17/2023,I can't wait to be out of this. This breaks me...,Skip,fairLight,green,dry,brown,P453253,Vegan Milk Moisturizer,MILK MAKEUP,38
1,7278410286,4,1,NaN,0,0,0,3/17/2023,I use it every day and my skin feels so smooth...,Nice Product,NaN,brown,combination,blonde,P453253,Vegan Milk Moisturizer,MILK MAKEUP,38
2,1048276344,5,1,1.0,2,0,2,12/28/2022,I have combo skin and am currently battling br...,USE IT ON YOUR CUTICLES,lightMedium,brown,combination,NaN,P453253,Vegan Milk Moisturizer,MILK MAKEUP,38
3,1859993604,5,1,1.0,2,0,2,12/7/2022,I have very dry and sensitive skin. This has b...,NaN,NaN,blue,dry,blonde,P453253,Vegan Milk Moisturizer,MILK MAKEUP,38
4,35010137982,1,0,1.0,1,0,1,12/4/2022,It doesn't do it for me. If you're going to us...,Itâ€™s not for me and thatâ€™s ok.,NaN,brown,normal,NaN,P453253,Vegan Milk Moisturizer,MILK MAKEUP,38


In [5]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
# import io
# df = pd.read_csv(io.BytesIO(uploaded['review_data.csv']))

In [6]:
from getpass import getpass
openai_key = getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


In [7]:
# Initialize LLM
llm = ChatOpenAI(openai_api_key=openai_key, temperature=0, model="gpt-3.5-turbo")

<ipython-input-7-859ad6bea186>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=openai_key, temperature=0, model="gpt-3.5-turbo")


In [9]:
prompt = "what is intermitten fasting explained in single sentence?"
response = llm.invoke(prompt)
print(response)

content='Intermittent fasting is a pattern of eating that alternates between periods of fasting and eating.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 18, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8df9ebf2-96b1-4fcd-9122-c08d40bfb08b-0'


In [10]:
# Define output schema
class ReviewAnalysis(BaseModel):
    summary: str = Field(description="Summary of the review in 1–2 sentences.")
    highlights: List[str] = Field(description="2–3 key points (positive, negative, or mixed).")
    sentiment: str = Field(description="Sentiment of the review: Positive / Neutral / Negative.")
    emotions: List[str] = Field(description="2–3 emotions expressed in the review.")
    business_insights: str = Field(description="Business implications based on the review.")
    recommended_action: str = Field(description="Suggested business action based on review.")
    email: str = Field(description="Suggested response email to the customer.")

parser = PydanticOutputParser(pydantic_object=ReviewAnalysis)

# Prompt template
prompt_template = """
You are a customer review analyst for a consumer-facing business.
Your task is to analyze customer reviews and extract actionable insights for business improvement and customer engagement.

Given the review below, please provide the following structured output:

1. Summary (1–2 sentences)
2. Key Points (2–3 bullets)
3. Sentiment Classification: Positive / Neutral / Negative
4. Emotions Detected (2–3 words)
5. Business Insights (1–2 sentences)
6. Recommended Action (optional)
7. Suggested Response Email

{format_instructions}

Review:
{review}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["review"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [11]:
# LCEL Chain
chain = prompt | llm | parser

In [12]:
# Analyze reviews from DataFrame
def analyze_reviews(data: pd.DataFrame, text_column: str = "review_text") -> pd.DataFrame:
    results = []
    for review in data[text_column]:
        try:
            structured_output = chain.invoke({"review": review})
            results.append(structured_output.dict())
        except Exception as e:
            print("Error processing review:", e)
            results.append({
                "summary": None,
                "highlights": None,
                "sentiment": None,
                "emotions": None,
                "business_insights": None,
                "recommended_action": None,
                "email": None
            })
    return pd.concat([data, pd.DataFrame(results)], axis=1)

In [15]:
analyzed_df = analyze_reviews(df)
analyzed_df.head()

<ipython-input-12-723c3c8183a4>:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  results.append(structured_output.dict())
<ipython-input-12-723c3c8183a4>:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  results.append(structured_output.dict())
<ipython-input-12-723c3c8183a4>:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  results.append(structured_output.dict())
<ipython-input-12-723c3c8183a4>:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instea

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,...,product_name,brand_name,price_usd,summary,highlights,sentiment,emotions,business_insights,recommended_action,email
0,9334422659,3,0,NaN,0,0,0,3/17/2023,I can't wait to be out of this. This breaks me...,Skip,...,Vegan Milk Moisturizer,MILK MAKEUP,38,Customer finds the product to be subpar compar...,"[Causes breakouts, Does not work well under ma...",Negative,"[Disappointment, Frustration]",The product needs improvement to meet customer...,Consider reformulating the product to address ...,"Dear Customer, Thank you for sharing your feed..."
1,7278410286,4,1,NaN,0,0,0,3/17/2023,I use it every day and my skin feels so smooth...,Nice Product,...,Vegan Milk Moisturizer,MILK MAKEUP,38,Effective moisturizer but heavy and thick.,"[Skin feels smooth, Product is heavy and thick...",Neutral,"[Satisfaction, Disappointment]",Customers appreciate the effectiveness of the ...,Consider offering a lighter version of the moi...,"Dear Customer, Thank you for your feedback on ..."
2,1048276344,5,1,1.0,2,0,2,12/28/2022,I have combo skin and am currently battling br...,USE IT ON YOUR CUTICLES,...,Vegan Milk Moisturizer,MILK MAKEUP,38,Versatile product that works well on nails but...,"[Effective for healing and repairing cuticles,...",Positive,"[Satisfaction, Preference]",Customers appreciate the product's effectivene...,Consider marketing the product as a nail and c...,"Dear Customer, Thank you for sharing your expe..."
3,1859993604,5,1,1.0,2,0,2,12/7/2022,I have very dry and sensitive skin. This has b...,NaN,...,Vegan Milk Moisturizer,MILK MAKEUP,38,"Effective for dry and sensitive skin, not suit...","[Effective for dry and sensitive skin, Reduces...",Positive,"[Satisfaction, Relief]",The product is well-suited for customers with ...,Consider marketing the product specifically to...,"Dear Customer, Thank you for sharing your expe..."
4,35010137982,1,0,1.0,1,0,1,12/4/2022,It doesn't do it for me. If you're going to us...,Itâ€™s not for me and thatâ€™s ok.,...,Vegan Milk Moisturizer,MILK MAKEUP,38,Customer found the product lacking in moisturi...,"[Product lacks slip and moisturizing effect, N...",Negative,"[Disappointed, Neutral]",The product needs improvement in providing sli...,Consider reformulating the product to enhance ...,"Dear Customer, Thank you for your feedback. We..."


In [17]:
analyzed_df.to_csv("/content/review_analysis_results.csv", index=False)